# BIAS-PROFS と InterProデータセットのラベルCSVおよびグラデーション画像を作成する

## 1. ライブラリの読み込み

In [24]:
import csv
import math

import Bio.Seq
import numpy as np
import os

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

from tqdm import tqdm
from PIL import Image

In [2]:
aas = list("ACDEFGHIKLMNPQRSTVWY")
exclusion_bases = list("BJOUXZ")

def check_valid_seq(seq):
    """
    有効なアミノ酸配列かどうかを返す関数
    :param seq: タンパク質配列
    :return: 有効なタンパク質ならば True
    """

    # 配列長が1000以上なら除外
    if len(seq) >= 1000:
        return False

    # "BJOUXZ" が含まれていたら除外
    for base in exclusion_bases:
        if base in seq:
            return False

    return True

## 2. BIAS-PROFS から読み込み

In [3]:
all_seq_data = {}

In [4]:
biasprofs_data_dir = "../data/gds_dataset"

biasprofs_filenames = [os.path.join(biasprofs_data_dir, file) for file in os.listdir(biasprofs_data_dir) if file.endswith(".txt")]  # .txt のみを取得

biasprofs_filenames.sort()  # 一応ソート
len(biasprofs_filenames)

biasprofs_seq_data = {}

In [5]:
for filename in tqdm(biasprofs_filenames):
    class_name = os.path.basename(filename).split("Class")[1][0]

    with open(filename, "r", encoding="utf-8") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            seq = str(record.seq)

            if check_valid_seq(seq):
                if seq not in biasprofs_seq_data:
                    biasprofs_seq_data[seq] = set()

                if seq not in all_seq_data:
                    all_seq_data[seq] = set()

                biasprofs_seq_data[seq].add(class_name)
                all_seq_data[seq].add(class_name)

100%|██████████| 87/87 [00:00<00:00, 1463.50it/s]


## 3. InterPro データセットから読み込み

In [6]:
interpro_data_dir = "../data/interpro"

interpro_filenames = [os.path.join(interpro_data_dir, file) for file in os.listdir(interpro_data_dir) if file.endswith(".fasta")]  # .txt のみを取得

interpro_filenames.sort()  # 一応ソート
len(interpro_filenames)

interpro_seq_data = {}

In [7]:
for filename in tqdm(interpro_filenames):
    class_name = os.path.basename(filename).split("_")[0][0]

    with open(filename, "r", encoding="utf-8") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            seq = str(record.seq)

            if check_valid_seq(seq):
                if seq not in interpro_seq_data:
                    interpro_seq_data[seq] = set()

                if seq not in all_seq_data:
                    all_seq_data[seq] = set()

                interpro_seq_data[seq].add(class_name)
                all_seq_data[seq].add(class_name)

100%|██████████| 7/7 [00:00<00:00, 480.16it/s]


In [8]:
print(len(biasprofs_seq_data))
print(len(interpro_seq_data))
print(len(all_seq_data))

4198
2667
5821


## 4. 2クラス以上に属している配列を除外する

In [17]:
all_seq_data_sorted = sorted(all_seq_data.items(), key=lambda x:x[1])
exclusion_duplicate = {key: list(value)[-1] for key, value in all_seq_data.items() if len(value) == 1 or "D" in value or "E" in value}
exclusion_duplicate_sorted = sorted(exclusion_duplicate.items(), key=lambda x:x[1])

print(len(all_seq_data_sorted))
print(len(exclusion_duplicate_sorted))

5821
5805


## 5. CSV に結果を保存

In [20]:
id = 0

with open("../data/my-gpcr-with-duplicate.csv", "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)

    for key, value in tqdm(all_seq_data_sorted):
        id += 1
        writer.writerow([id, len(value), sorted(list(value)), key])

id = 0

with open("../data/my-gpcr-labels.csv", "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)

    for key, value in tqdm(exclusion_duplicate_sorted):
        id += 1
        writer.writerow([id, value, key])

print("\033[32m✔ Completed!\033[0m")

100%|██████████| 5805/5805 [00:00<00:00, 204305.76it/s]

✔ Completed!


## 6. 1つのFASTAファイルにまとめる

In [26]:
id = 0

with open("../data/my-gpcr.fasta", "w", encoding="utf-8") as handle:
    for seq, class_name in exclusion_duplicate_sorted:
        id += 1
        record = SeqRecord(Bio.Seq.Seq(seq), str(id), "", class_name)
        SeqIO.write(record, handle, "fasta")

## 7. 内訳を表示

In [27]:
total_aa = 0
class_count = {class_name: 0 for class_name in list("ABCDEF")}
aa_count = {aa: 0 for aa in aas}
lens = []

for seq, class_name in exclusion_duplicate_sorted:
    total_aa += len(seq)
    class_count[class_name] += 1

    for aa in seq:
        aa_count[aa] += 1

    lens.append(len(seq))


print("sequence count:", len(exclusion_duplicate_sorted))
print("total aa:", total_aa)
print("class_count:", class_count)
print("aa count:", aa_count)
print("avg len:", np.mean(lens))
print("min len:", min(lens))
print("max len:", max(lens))

sequence count: 5805
total aa: 2490908
class_count: {'A': 4204, 'B': 360, 'C': 1144, 'D': 17, 'E': 15, 'F': 65}
aa count: {'A': 176217, 'C': 77867, 'D': 80622, 'E': 90720, 'F': 149251, 'G': 130187, 'H': 57289, 'I': 173274, 'K': 100259, 'L': 294233, 'M': 69798, 'N': 102197, 'P': 117817, 'Q': 75409, 'R': 117039, 'S': 204202, 'T': 150010, 'V': 189740, 'W': 45060, 'Y': 89717}
avg len: 429.0969853574505
min len: 60
max len: 999


## 8. グラフ表示画像の作成

In [30]:
amino_vec = {
    "A": {"x":  0.99019942, "y": 2.29554027},
    "R": {"x": -7.38605815, "y": 1.30236133},
    "N": {"x": -4.01061596, "y": 2.98579296},
    "D": {"x": -2.04788011, "y": 1.43394109},
    "C": {"x":  2.18212092, "y": 2.05872491},
    "Q": {"x": -4.99366066, "y": 3.32616193},
    "E": {"x": -3.73512536, "y": 3.32397933},
    "G": {"x": -0.04937042, "y": 0.49755659},
    "H": {"x": -4.48215043, "y": 3.98877519},
    "I": {"x":  5.41644264, "y": 0.95506498},
    "L": {"x":  5.11141138, "y": 2.03063381},
    "K": {"x": -6.26454053, "y": 3.12338469},
    "M": {"x":  3.58295155, "y": 4.81273916},
    "F": {"x":  5.09870182, "y": 4.03153069},
    "P": {"x": -2.46839549, "y": 4.91497952},
    "S": {"x": -0.17443449, "y": 2.99492447},
    "T": {"x": -0.58046457, "y": 4.96619179},
    "W": {"x": -0.32567719, "y": 6.99241978},
    "Y": {"x": -2.00762263, "y": 6.70592659},
    "V": {"x":  4.82962913, "y": 1.29409523},
}

image_save_dir = "../graphs/my-gpcr"
os.makedirs(image_save_dir, exist_ok=True)

size = 224
padding = 3
linewidth = 2

In [31]:
def map_range(x, a, b, c, d):
    """
    変数 $x$ を 範囲 [$a$, $b$] から [$c$, $d$] にリマップします
    :param x: 変換する対象
    :param a: 変換前の下端
    :param b: 変換前の上端
    :param c: 変換後の下端
    :param d: 変換後の上端
    :return: リマップした値
    """
    return (x - a) / (b - a) * (d - c) + c


def draw_point(canvas, x: int, y: int, width: int, intensity: int):
    """
    点を描画します
    :param canvas: numpy の2次元配列
    :param x: 座標 $x$
    :param y: 座標 $y$
    :param width: 点の幅
    :param intensity: 点の白色の強度
    :return:
    """

    size = canvas.shape[0]

    for i in range(-width, width + 1):
        yi = y + i

        if 0 <= yi < size:
            for j in range(-width, width + 1):
                xi = x + j

                if 0 <= xi < size:
                    r = math.sqrt(i ** 2 + j ** 2)  # 中心からの距離

                    if r <= width:  # 半径内部かどうかを判定
                        val = int(intensity * (1 - r / width))  # 直線的に色を変化
                        # val = int((intensity / (r_0 + 1)) - 1)  # 中心から反比例するように色を変化

                        if val > canvas[yi, xi]:  # 画素よりも大きい値であれば上書き
                            canvas[yi, xi] = val


def drawline_with_bresenham_algorithm(canvas, x0: int, y0: int, x1: int, y1: int, width: int, intensity: int):
    """
    ブレゼンハムのアルゴリズムで2点間の直線を描画します
    :param canvas: numpy の2次元配列
    :param x0: 点 $x_0$
    :param y0: 点 $y_0$
    :param x1: 点 $x_1$
    :param y1: 点 $y_1$
    :param width 直線の幅
    :return: None
    """

    # ここで float を int に変換
    x0 = round(x0)
    y0 = round(y0)
    x1 = round(x1)
    y1 = round(y1)

    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        draw_point(canvas, x0, y0, width, intensity)

        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * err

        if e2 > -dy:
            err -= dy
            x0 += sx

        if e2 < dx:
            err += dx
            y0 += sy

    draw_point(canvas, x1, y1, width, intensity)


def generate_graph(seq, accession_number, size, padding, width):
    """
    アミノ酸配列からグラフ表示画像を作成します
    :param seq: アミノ酸配列
    :param accession_number: アクセッション番号
    :param size: 画像サイズ
    :param padding: 画像のパディング
    :param width: 線幅
    :return:
    """

    x = y = 0
    x_min = x_max = 0
    y_min = y_max = 0

    points = [{"x": 0, "y": 0}]

    for c in seq:
        x += amino_vec[c]["x"]
        y += amino_vec[c]["y"]  # 人間が見るXY平面にする

        points.append({"x": x, "y": y})

        x_min, x_max = min(x, x_min), max(x, x_max)
        y_min, y_max = min(y, y_min), max(y, y_max)

    mapped_points = [
        {
            "x": map_range(point["x"], x_min, x_max, padding, (size - padding)),
            "y": size - map_range(point["y"], y_min, y_max, padding, (size - padding))  # xy 平面にするために反転
        }
        for point in points
    ]

    canvas = np.zeros((size, size), dtype=np.uint8)
    n_segments = len(mapped_points) - 1

    for i in range(n_segments):
        start, end = mapped_points[i], mapped_points[i + 1]

        progress = i / n_segments
        intensity = int(255 * progress)
        # draw.line([(start["x"], start["y"]), (end["x"], end["y"])], fill=255, width=3)
        drawline_with_bresenham_algorithm(canvas, start["x"], start["y"], end["x"], end["y"], width, intensity)

    img = Image.fromarray(canvas, mode="L")  # numpy 配列から画像を生成
    img.save(f"./{image_save_dir}/{accession_number}.png")

In [33]:
num = 0

with open("../data/my-gpcr.fasta", "r") as handle:
    for record in tqdm(SeqIO.parse(handle, "fasta")):
        seq = str(record.seq)

        if check_valid_seq(seq):
            num += 1
            generate_graph(seq, num, size, padding, linewidth)

print("\033[32m✔ Completed!\033[0m")

5805it [01:38, 58.89it/s]

✔ Completed!
